# Classification sample code
The goal is checking an input token is positive or negative

## Prerequisite
* Download and uncompress the file to get `entity_vector.model.bin` from http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/  

* Move `entity_vector.model.bin` to `data/embeddings/` folder

# Prepare data

In [2]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models.keyedvectors import KeyedVectors
from keras.layers import Dense,Activation,Dropout
from keras.models import Sequential

SIZE_OF_W2V = 200
W2V = KeyedVectors.load_word2vec_format('../data/embeddings/entity_vector.model.bin', binary=True)

def prepare_y(y):
    if int(y) == 2:
        return [1,0]
    else:
        return [0,1]
    
def prepare_x(x):
    try:
        return W2V[x]
    except:
        return [0] * SIZE_OF_W2V

x_data = []
y_data = []

with open('../data/training/sentiment.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        x_data.append(prepare_x(row["x"]))
        y_data.append(prepare_y(row["y"]))
        
x_data = np.array(x_data)
y_data = np.array(y_data)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

print(x_train.shape)
print(y_train.shape)

(18, 200)
(18, 2)


# Train model

In [3]:
def create_model(input_length):
    model = Sequential()
    model.add(Dense(SIZE_OF_W2V, input_dim=SIZE_OF_W2V))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

model = create_model(len(x_train))
model.fit(x_train, y_train, batch_size=1, epochs=10, verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/10
18/18 [==============================] - 0s 18ms/step - loss: 0.7904 - accuracy: 0.6389
Epoch 2/10
18/18 [==============================] - 0s 2ms/step - loss: 0.4763 - accuracy: 0.7778
Epoch 3/10
18/18 [==============================] - 0s 3ms/step - loss: 0.2291 - accuracy: 0.9722
Epoch 4/10
18/18 [==============================] - 0s 3ms/step - loss: 0.1571 - accuracy: 0.9722
Epoch 5/10
18/18 [==============================] - 0s 4ms/step - loss: 0.2348 - accuracy: 0.9444
Epoch 6/10
18/18 [==============================] - 0s 4ms/step - loss: 0.0997 - accuracy: 1.0000
Epoch 7/10
18/18 [==============================] - 0s 3ms/step - loss: 0.1092 - accuracy: 0.9722
Epoch 8/10
18/18 [==============================] - 0s 2ms/step - loss: 0.0778 - accuracy: 1.0000
Epoch 9/10
18/18 [==============================] - 0s 3ms/step - loss: 0.0609 - accuracy: 1.0000
Epoch 10/10
18/18 [===

# Test

In [4]:
results = model.evaluate(x_test, y_test)
print(f"error: {results[0]}, accuracy: {results[1]}")

5/5 [==============================] - 0s 9ms/step
error: 0.7024508714675903, accuracy: 0.6000000238418579


# Predict

In [5]:
def translate_y(y):
    print(y)
    if y.flat[0] >= y.flat[1]:
        return "negative"
    else:
        return "postive"
        
def do_prediction(x):
    label = translate_y(model.predict(np.array(prepare_x(x)).reshape(1,-1)))
    print(f"{x} is {label} \n")

do_prediction("好い")
do_prediction("邪悪")
do_prediction("ナイス")
do_prediction("堕落")

[[0.02864467 0.96477437]]
好い is postive 

[[0.99978644 0.00158434]]
邪悪 is negative 

[[0.00697057 0.991812  ]]
ナイス is postive 

[[0.99359775 0.00222857]]
堕落 is negative 

